In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from sklearn.metrics import classification_report


import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import scores as s

from pprint import pprint

In [2]:
a00, a01 = h.import_desired_data("A", "15T")

w3 = w.Window_Detect2(a01)
w3.run_analyis_and_guess(w.make_ewm_smooth, )
# w3.analyze_window_change(w.make_ewm_smooth)

In [3]:
w3.plot_guesses(make_guesses=True, timedelta=15, z=1)

In [4]:
w3.plot_guesses(make_guesses=True, timedelta=15, z=2)

In [5]:
len(w3.window.unique()) > 1

True

In [6]:
metrics = {}

for z in [1, 2]:
    w3.run_guess(z=z)
    reset_interp = w3.interp_guess.reset_index()
    choices = reset_interp[0]
    truth = w3.window

    s1 = s.Scores(exp=a01, choices=choices, guess_times=w3.guess_times)
    metrics[f"z={z}"] = {}
    metrics[f"z={z}"]["drdr"] = s1.calc_drdr_metrics()
    metrics[f"z={z}"]["standard"] = classification_report(truth, choices, output_dict=True)


In [7]:
w3.guess_times

2     2022-07-20 07:45:00
41    2022-07-20 17:30:00
60    2022-07-20 22:15:00
61    2022-07-20 22:30:00
102   2022-07-21 08:45:00
105   2022-07-21 09:30:00
137   2022-07-21 17:30:00
138   2022-07-21 17:45:00
140   2022-07-21 18:15:00
157   2022-07-21 22:30:00
189   2022-07-22 06:30:00
193   2022-07-22 07:30:00
194   2022-07-22 07:45:00
196   2022-07-22 08:15:00
198   2022-07-22 08:45:00
199   2022-07-22 09:00:00
205   2022-07-22 10:30:00
206   2022-07-22 10:45:00
223   2022-07-22 15:00:00
225   2022-07-22 15:30:00
255   2022-07-22 23:00:00
291   2022-07-23 08:00:00
Name: DateTime, dtype: datetime64[ns]

In [8]:
s1.report_scores()

,results
hits/guesses,0.227
hits/actions,0.385
(hits + near hits)/guesses,0.545
(hits + near hits)/actions,0.923
misses/guesses,0.455
number of actions,13.000
number of guesses,22.000
hit,5.000
near_hit,7.000
miss,10.000


In [9]:
pprint(metrics, width=20, )

{'z=1': {'drdr': {'# false open instance': 0,
                  '# true open instance': 4,
                  'bounded acc': 0.5,
                  'false open time': Timedelta('1 days 02:00:00'),
                  'true open time': Timedelta('2 days 22:15:00'),
                  'unbounded acc': -2.19},
         'standard': {'0.0': {'f1-score': 0.6848484848484848,
                              'precision': 0.6608187134502924,
                              'recall': 0.710691823899371,
                              'support': 159},
                      '1.0': {'f1-score': 0.7636363636363637,
                              'precision': 0.7850467289719626,
                              'recall': 0.7433628318584071,
                              'support': 226},
                      'accuracy': 0.7298701298701299,
                      'macro avg': {'f1-score': 0.7242424242424242,
                                    'precision': 0.7229327212111275,
                                    'reca